<a href="https://colab.research.google.com/github/rogeriomoreirajr/dataviz/blob/master/BTPEsquisa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from oauth2client.client import GoogleCredentials
from google.colab import auth
auth.authenticate_user()

# from IPython import Markdown, display

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

gc = gspread.authorize(GoogleCredentials.get_application_default())

sheet = gc.open_by_key('1o3gYymxUmQACupnBF1N7JOFty0U51MNdyJtGtdANmrk')
atual = sheet.worksheet('SORTEIO BTCAST 2019')
dfg = get_as_dataframe(atual)

In [114]:
dic_cols = {
 'Qual o sexo?':'sexo',
 'Qual a sua idade?':'idade',
 'Qual é a sua escolaridade?':'escolaridade',
 'Qual é a sua ocupação atual?':'ocupacao',
 'Qual o seu estado?':'estado',
 'Você mora na...':'capital-interior',
 'Qual sua afiliação religiosa?':'afiliacao',
 'Qual sua renda mensal em salários mínimos?':'renda',
 'Com essa renda, você auxilia nas despesas de casa?':'rendaAjuda',
 'Tem o costume de comprar on-line?':'compraOnline',
 'Já comprou alguma coisa por indicação do blog/podcast/vlog?':'compraPodcast',
 'De 1 a 10 joinhas, qual a credibilidade do BiboTalk (blog, podcast e vlog)?':'credibilidade',
 'Como você conheceu o BTCast?':'conheceu',
 'Qual é a duração ideal de um BTCast?':'duracao',
 'Como você escuta podcast?':'escutaComo',
 'Normalmente, você escuta o podcast...':'escutaQuando',
 'De qual série do BTCasts você mais gosta?':'serie',
 'Qual BTCast precisa de uma continuação?':'continuacao',
 'Quais podcasts da casa você escuta?':'podcastQual',
 'Você acha que a gente publica muitos episódios por semana?':'episodiosMuitos',
 'Você teria interesse em participar de um BTDay na sua cidade?':'btday',
 'Já comprou na Amazon pelo nosso link?':'compraAmazon',
 'Qual é a duração ideal de um BTVlog?':'btvlogDuracao',
 'O que você deseja ver no BTVlog?':'btvlogTema',
 'Token':'token'
}

df = dfg[dic_cols.keys()]
df.columns = dic_cols.values()
df.compraAmazon = df.compraAmazon==1.0

from pandas.api.types import CategoricalDtype

cat = CategoricalDtype(
    categories = [
                    'Acima de 20 salários mínimos',
                    'Entre 10 e 20 salários mínimos',
                    'Entre 4 e 10 salários mínimos',
                    'Entre 2 e 4 salários mínimos',
                    'Ainda ganho mesada',
                    'Não tenho renda alguma',
                    'não quero responder',
                  'Total'
                ], ordered = True
)

df.renda = df.renda.astype(cat)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


# Quais as perguntas?

- Qual o sexo?
- Qual o estado? E região? Capital ou interior?
- Afiliação religiosa
- Renda. Compra online? Compra por podcast? Compra Amazon?


In [125]:
# Sexo

tb = df.groupby('sexo').token.count()
sexo = tb/tb.sum()

pd.DataFrame(sexo).style.format('{:.2%}')

,token
sexo,
Feminino,25.39%
Masculino,74.61%


In [124]:
# Estado
"""
Posso usar aqui o "regiao" para o caso de São Paulo. 16% na capital, 
e outros 16% no interior do Estado
"""

tb = df.groupby('estado').token.count()
estados = (tb/tb.sum()).sort_values(ascending=False)

pd.DataFrame(estados).style.format('{:.2%}')

,token
estado,
São Paulo (SP),31.77%
Rio de Janeiro (RJ),13.83%
Minas Gerais (MG),8.04%
Paraná (PR),6.71%
Santa Catarina (SC),5.90%
Rio Grande do Sul (RS),4.92%
Pernambuco (PE),4.34%
Bahia (BA),3.12%
Ceará (CE),3.07%


In [122]:
# Afiliação religiosa
"""
Um quarto dos ouvintes são batistas, enquanto que 16,60 são da Assembleia de Deus
"""
tb = df.groupby('afiliacao').token.count()
afiliacao = pd.DataFrame((tb/tb.sum()).sort_values(ascending=False))
afiliacao.style.format('{:.2%}')

,token
afiliacao,
Batista,25.91%
Assembleia de Deus,16.60%
Presbiteriano,12.32%
Pentecostal Outras,11.45%
Comunidade Independente Pentecostal,7.35%
Comunidade Independente Reformada,6.77%
"Cristão, sem afiliação denominacional",4.40%
Neopentecostal Outras,3.24%
Quadrangular,2.78%


In [121]:
"""
faixa de renda | Compra Online? | Compra Podcast? | Comprou Amazon já?
"""

# Renda
renda = df.groupby('renda').token.count().rename('pessoas')
compraOnline = df.query('compraOnline').groupby('renda').token.count().rename('compraOnline')
compraPodcast = df.query('compraPodcast').groupby('renda').token.count().rename('compraPodcast')
compraAmazon = df.query('compraAmazon').groupby('renda').token.count().rename('compraAmazon')

tb = pd.concat([renda, compraOnline, compraPodcast, compraAmazon], axis=1)

total = tb.sum()

tb.loc['Total'] = total

for coluna in ['compraOnline', 'compraPodcast', 'compraAmazon']:
    tb[coluna] = tb[coluna]/total.sum()

    # Embelezar
    tb[coluna] = tb[coluna].apply(lambda x: '{:.2%}'.format(x))

,pessoas,compraOnline,compraPodcast,compraAmazon
renda,,,,
Acima de 20 salários mínimos,8,0.12%,0.12%,0.06%
Entre 10 e 20 salários mínimos,88,1.65%,1.48%,0.94%
Entre 4 e 10 salários mínimos,329,6.16%,5.03%,3.36%
Entre 2 e 4 salários mínimos,922,16.45%,12.98%,6.81%
Ainda ganho mesada,75,1.07%,0.79%,0.25%
Não tenho renda alguma,193,2.84%,2.15%,0.83%
não quero responder,114,1.94%,1.31%,0.50%
Total,1729,30.23%,23.84%,12.74%
